# Kevin Soffa
## -Projeto de automação de busca de preços mais barato
## -Enviar notificações

#### Passo 1: importar planilha para o Python

#### Passo 2: Percorrer a planilha
Para cada linha: Pegar preço Amazon, Lojas Americanas, Magazine Luiza.
Registrar o menor preço, local onde está o menor preço.

#### Passo 3: Salvar planilha

#### Passo 4: Enviar e-mail
Se o preço estivar com + de 20% de desconto enviar notificação por e-mail.

In [64]:
# Importando biblioteca

import pandas as pd
from selenium import webdriver
import time


# Lendo a planilha
produtos = pd.read_excel('Produtos.xlsx')
display(produtos)

,Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,Apple iPhone 12 Pro Max 256GB,https://www.amazon.com.br/Novo-Apple-iPhone-Az...,https://www.americanas.com.br/produto/22909633...,https://www.magazineluiza.com.br/iphone-12-pro...,8899,NaN,NaN
1,"Smart TV LED 50"" UHD 4K Samsung",https://www.amazon.com.br/Samsung-Crystal-50AU...,https://www.americanas.com.br/produto/30695061...,https://www.magazineluiza.com.br/smart-tv-50-c...,2799,NaN,NaN
2,Apple MacBook Air 13.3,https://www.amazon.com.br/Apple-MacBook-13-3-C...,https://www.americanas.com.br/produto/25450804...,https://www.magazineluiza.com.br/macbook-air-1...,7839,NaN,NaN


In [76]:
def transformar_texto(texto):
    return float(texto.replace('R$', '').replace('.', '').replace(',','.'))

In [91]:
# Entrar no Chrome
driver = webdriver.Chrome()

# Não mostrar tela do Google Chrome
#driver.set_window_position(-1100,0)


enviar_email = False
desconto_min = 0.2


# Percorrendo as linhas e colunas
for i, linha in produtos.iterrows():
    
    # Pegar preço Amazon
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_id('priceblock_ourprice').text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        try:
            preco_amazon = driver.find_element_by_css_selector('#all-offers-display span.a-size-base')
            preco_amazon = transformar_texto(preco_amazon)
        except:
            print('Produto {} não disponível na Amazon'.format(linha['Produto']))
            preco_amazon = linha['Preço Original'] * 3
    



    # Pegar preço da Loja Americanas
    driver.get(linha['Lojas Americanas'])
    time.sleep(10)
    try:
        preco_lojasamericanas = driver.find_element_by_class_name('src__BestPrice-sc-1jvw02c-5').text
        preco_lojasamericanas = transformar_texto(preco_lojasamericanas)
    except:
        print('Produto {} não disponível na Lojas Americanas'.format(linha['Produto']))
        preco_lojasamericanas = linha['Preço Original'] * 3



    # Pegar o preço da Magalu
    driver.get(linha['Magazine Luiza'])
    try:
        preco_magazine = driver.find_element_by_class_name('price-template__text').text
        preco_magazine = transformar_texto(preco_magazine)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Produto']))
        preco_magazine = linha['Preço Original'] * 3
        
    
    print('=-'*30)
    print(preco_amazon, preco_lojasamericanas, preco_magazine)
   

    
    preco_original = linha['Preço Original']
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lojasamericanas, 'Lojas Americanas'), (preco_magazine, 'Magazine Luiza'), (preco_original, 'Preço original')]
    
    # Ordenando em ordem crescente
    lista_precos.sort()
    
    print('--'*30)
    print(lista_precos[0])
    print('=-'*30)
    
    
    # Adicionando os preços na planilha
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos.loc[i, 'Local'] = lista_precos[0][1]

    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
8795.0 8555.07 8555.07
------------------------------------------------------------
(8555.07, 'Lojas Americanas')
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
5027.93 2944.05 2749.0
------------------------------------------------------------
(2749.0, 'Magazine Luiza')
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
7787.43 8989.9 9679.55
------------------------------------------------------------
(7787.43, 'Amazon')
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [69]:
# Preenchendo as colunas Preço atula e Local

display(produtos)

,Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,Apple iPhone 12 Pro Max 256GB,https://www.amazon.com.br/Novo-Apple-iPhone-Az...,https://www.americanas.com.br/produto/22909633...,https://www.magazineluiza.com.br/iphone-12-pro...,8899,8555.07,Lojas Americanas
1,"Smart TV LED 50"" UHD 4K Samsung",https://www.amazon.com.br/Samsung-Crystal-50AU...,https://www.americanas.com.br/produto/30695061...,https://www.magazineluiza.com.br/smart-tv-50-c...,2799,2749.00,Magazine Luiza
2,Apple MacBook Air 13.3,https://www.amazon.com.br/Apple-MacBook-13-3-C...,https://www.americanas.com.br/produto/25450804...,https://www.magazineluiza.com.br/macbook-air-1...,7839,7787.43,Amazon


In [73]:
# Salvar arquivo no pc
produtos.to_excel('Produto.xlsx')

# Variável desconto
desconto_min = 0.2

# Enviar email
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

enviar_email = True
if enviar_email:
    #filtrar tabela
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]
    
    mail = outlook.CreateItem(0)
    mail.To = 'kevin@gmail.com'
    mail.Subject = f'Produto(s) Encontrado(s) com mais de {desconto_min:.0%} de Desconto'
    #mail.Body = 'Texto do E-mail'
    
    # Corpo do e-mail em HTML
    mail.HTMLBody = f'<p>Esses são os produtos com mais de {desconto_min:.0%} de DESCONTO</p>{tabela_filtrada.to_html()}'
    
    mail.Send()
    
    
print('FIM DA ANÁLISE')
driver.quit()


FIM DA ANÁLISE
